In [4]:
# 4.1 手动遍历迭代器
'''
问题
你想遍历一个可迭代对象中的所有元素，但是却不想使用 for 循环。
解决方案
为了手动的遍历可迭代对象，使用 next() 函数并在代码中捕获 StopIteration 异
常。比如，下面的例子手动读取一个文件中的所有行：
'''
def manual_iter():
    with open('1_3.txt',encoding='utf-8') as f:
        try:
            while True:
                line = next(f)
                print(line, end='')
        except StopIteration:
            pass

def manual_iter1():
    '''
    通常来讲，StopIteration 用来指示迭代的结尾。然而，如果你手动使用上面演示
    的 next() 函数的话，你还可以通过返回一个指定值来标记结尾，比如 None 。下面是
    示例：
    :return:
    '''
    with open('1_3.txt',encoding='utf-8') as f:
        while True:
            line = next(f,None)
            if line is None:
                break
            print(line,end="")

'''
讨论
大多数情况下，我们会使用 for 循环语句用来遍历一个可迭代对象。但是，偶尔也
需要对迭代做更加精确的控制，这时候了解底层迭代机制就显得尤为重要了。
'''
# manual_iter1()

'\n讨论\n大多数情况下，我们会使用 for 循环语句用来遍历一个可迭代对象。但是，偶尔也\n需要对迭代做更加精确的控制，这时候了解底层迭代机制就显得尤为重要了。\n'

In [6]:
# 4.2 代理迭代
'''
问题
你构建了一个自定义容器对象，里面包含有列表、元组或其他可迭代对象。你想直
接在你的这个新容器对象上执行迭代操作。
解决方案
实际上你只需要定义一个 __iter__() 方法，将迭代操作代理到容器内部的对象上
去。比如：
'''
class Node:
    
    def __init__(self,value):
        self._value = value
        self._children = []
        
        
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    
    def add_child(self,node):
        self._children.append(node)
        
    
    def __iter__(self):
        return  iter(self._children)
    

if __name__ == '__main__':
    root = Node(0)
    child1 = Node(1)
    child2 = Node(2)
    root.add_child(child1)
    root.add_child(child2)
    
    for ch in root:
        print(ch)
    
        

Node(1)
Node(2)


In [19]:
# 4.3 使用生成器创建新的迭代模式
'''
问题
你想实现一个自定义迭代模式，跟普通的内置函数比如 range() , reversed() 不
一样。
解决方案
如果你想实现一种新的迭代模式，使用一个生成器函数来定义它。下面是一个生产
某个范围内浮点数的生成器：
'''

def frange(start,stop,increment):
    x = start 
    while x < stop:
        yield x
        x += increment
        
# 为了使用这个函数，你可以用 for 循环迭代它或者使用其他接受一个可迭代对象的
# 函数 (比如 sum() , list() 等)。示例如下：
for n in frange(0,4,0.5):
    print(n)
    
list(frange(0,1,0.125))



# 一个函数中需要有一个 yield 语句即可将其转换为一个生成器。跟普通函数不同
# 的是，生成器只能用于迭代操作。
def countdown(n):
    print('Starting to count from', n)
    while n > 0:
        yield n
        n-=1
    print('Done!')
    
c = countdown(3)
next(c)
next(c)
next(c)

0
0.5
1.0
1.5
2.0
2.5
3.0
3.5
Starting to count from 3


1

In [21]:
# 4.4 实现迭代器协议
'''
问题
你想构建一个能支持迭代操作的自定义对象，并希望找到一个能实现迭代协议的
简单方法。
解决方案
目前为止，在一个对象上实现迭代最简单的方式是使用一个生成器函数。在 4.2 小
节中，使用 Node 类来表示树形数据结构。你可能想实现一个以深度优先方式遍历树形
节点的生成器。下面是代码示例：
'''
# 深度优先方式遍历树
class Node:
    
    def __init__(self,value):
        self._value = value
        self._children = []

    
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    
    def add_child(self,node):
        self._children.append(node)
        
        
    def __iter__(self):
        return iter(self._children)
    
    
    def depth_first(self):
        yield self
        for c in self:
            yield from c.depth_first()
            
            
if __name__ == '__main__':
    root = Node(0)
    child1 = Node(1)
    child2 = Node(2)
    root.add_child(child1) 
    root.add_child(child2)
    child1.add_child(Node(3))
    child1.add_child(Node(4))
    child2.add_child(Node(5))
            
    for ch in root.depth_first():
        print(ch)
        

# 如何使用一个关联迭代器类
# 重新实现 depth_first() 方法
class Node:
    def __init__(self,value):
        self._value = value
        self._children = []

    
    def __repr__(self):
        return 'Node({!r})'.format(self._value)
    
    
    def add_child(self,node):
        self._children.append(node)
        
        
    def __iter__(self):
        return iter(self._children)
    
    
    def depth_first(self):
        return DeprecationWarning(self)
    
    
class DepthFirstIterator(object):
    
    def __init__(self,start_node):
        self._children_iter = Node
        self._child_iter = None
        self._node = start_node
        
        
    def __iter__(self):
        return self
    
    
    def __next__(self):
        if self._child_iter is Node:
            self._child_iter = iter(self._node)
            return self._node
        
        elif self._child_iter:
            try:
                nextchild = next(self._child_iter)
                return nextchild
            except StopIteration:
                self._child_iter = Node
                return next(self)
        else:
            self._child_iter = next(self._child_iter).depth_first()
            return next(self)
            
            

Node(0)
Node(1)
Node(3)
Node(4)
Node(2)
Node(5)


In [29]:
# 4.5 反向迭代
'''
问题
你想反方向迭代一个序列
解决方案
使用内置的 reversed() 函数，比如：
'''
a = [1,2,3,4]
for x in reversed(a):
    print(x)
    
class Countdown3:
    
    def __init__(self,start):
        self.start = start
        
    def __iter__(self):
        '''
        定义一个反向迭代器可以使得代码非常的高效，因为它不再需要将数据填充到一
个列表中然后再去反向迭代这个列表
        :return: 
        '''
        n = self.start
        while n > 0:
            yield n
            n -= 1
            
    
    def __reversed__(self):
        n = 1
        while n <= self.start:
            yield n
            n += 1
            
for rr in reversed(Countdown3(30)):
    print(rr)
    
for rr in Countdown3(30):
    print(rr)
    

4
3
2
1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1


In [31]:
# 4.6 带有外部状态的生成器函数
'''
问题
你想定义一个生成器函数，但是它会调用某个你想暴露给用户使用的外部状态值。
解决方案
如果你想让你的生成器暴露外部状态给用户，别忘了你可以简单的将它实现为一
个类，然后把生成器函数放到 __iter__() 方法中过去。比如：
'''

from collections import deque

class linehistory:
    
    def __init__(self,lines,histlen=3):
        self.lines = lines
        self.history = deque(maxlen==histlen)
        
        
    def __iter__(self):
        for lineno, line in enumerate(self.lines,1):
            self.history.append((lineno,line))
            yield line
            
    
    def clear(self):
        self.history.clear()
        
with open('1_3.txt',encoding='utf-8') as f:
    lines = linehistory(f)
    for line in lines:
        if 'python' in line:
            for lineno,hline in lines.history:
                print('{}{}'.format(lineno,hline),end='')


FileNotFoundError: [Errno 2] No such file or directory: '1_3.txt'

In [36]:
# 4.7 迭代器切片
'''
问题
你想得到一个由迭代器生成的切片对象，但是标准切片操作并不能做到。
解决方案
函数 itertools.islice() 正好适用于在迭代器和生成器上做切片操作。比如：
'''
import itertools
def count(n):
    while True:
        yield n
        n += 1
        
c = count(0)
# c[10:20]
for x in itertools.islice(c,10,20):
    print(x)
    '''
    迭代器和生成器不能使用标准的切片操作，因为它们的长度事先我们并不知道 (并
且也没有实现索引)。函数 islice() 返回一个可以生成指定元素的迭代器，它通过遍
历并丢弃直到切片开始索引位置的所有元素。然后才开始一个个的返回元素，并直到切
片结束索引位置。
这里要着重强调的一点是 islice() 会消耗掉传入的迭代器中的数据。必须考虑到
迭代器是不可逆的这个事实。所以如果你需要之后再次访问这个迭代器的话，那你就得
先将它里面的数据放入一个列表中。
    '''

10
11
12
13
14
15
16
17
18
19


In [37]:
# 4.8 跳过可迭代对象的开始部分
'''
问题
你想遍历一个可迭代对象，但是它开始的某些元素你并不感兴趣，想跳过它们。
解决方案

itertools 模块中有一些函数可以完成这个任务。首先介绍的是 itertools.
dropwhile() 函数。使用时，你给它传递一个函数对象和一个可迭代对象。它会返
回一个迭代器对象，丢弃原有序列中直到函数返回 Flase 之前的所有元素，然后返回后
面所有元素。
'''
# 为了演示，假定你在读取一个开始部分是几行注释的源文件。比如


'\n问题\n你想遍历一个可迭代对象，但是它开始的某些元素你并不感兴趣，想跳过它们。\n解决方案\n\nitertools 模块中有一些函数可以完成这个任务。首先介绍的是 itertools.\ndropwhile() 函数。使用时，你给它传递一个函数对象和一个可迭代对象。它会返\n回一个迭代器对象，丢弃原有序列中直到函数返回 Flase 之前的所有元素，然后返回后\n面所有元素。\n'

In [53]:
# 4.9 排列组合的迭代
'''
问题
你想迭代遍历一个集合中元素的所有可能的排列或组合
解决方案
itertools 模 块 提 供 了 三 个 函 数 来 解 决 这 类 问 题。其 中 一 个 是 itertools.
permutations() ，它接受一个集合并产生一个元组序列，每个元组由集合中所有
元素的一个可能排列组成。也就是说通过打乱集合中元素排列顺序生成一个元组，比
如：
'''
items = ['a', 'b', 'c']
from itertools import permutations


# 它接受一个集合并产生一个元组序列，每个元组由集合中所有元素的一个可能排列组成。
# 也就是说通过打乱集合中元素排列顺序生成一个元组，
for p in permutations(items):
    print(p)
# 如果你想得到指定长度的所有排列，你可以传递一个可选的长度参数。就像这样：
for p in permutations(items,2):
    print(p)
    
    
# 使用 itertools.combinations() 可得到输入集合中元素的所有的组合。比如：
from itertools import  combinations
for c in combinations(items,2):
    # 对于 combinations() 来讲，元素的顺序已经不重要了。也就是说，组合 ('a','b') 跟 ('b', 'a') 其实是一样的 (最终只会输出其中一个
    print('\n',c)


# 函 数 itertools.combinations_with_replacement() 允许同一个元素被选择多次，比如： 
from itertools import  combinations_with_replacement
for c in combinations_with_replacement(items,2):
    print(c)
    

('a', 'b', 'c')
('a', 'c', 'b')
('b', 'a', 'c')
('b', 'c', 'a')
('c', 'a', 'b')
('c', 'b', 'a')
('a', 'b')
('a', 'c')
('b', 'a')
('b', 'c')
('c', 'a')
('c', 'b')

 ('a', 'b')

 ('a', 'c')

 ('b', 'c')
('a', 'a')
('a', 'b')
('a', 'c')
('b', 'b')
('b', 'c')
('c', 'c')


In [58]:
# 4.10 序列上索引值迭代
'''
问题
你想在迭代一个序列的同时跟踪正在被处理的元素索引。

解决方案
内置的 enumerate() 函数可以很好的解决这个问题
'''
my_list = ['a', 'b', 'c']
for idx, val in enumerate(my_list):
    print(idx,val)
    
for idx, val in enumerate(my_list,1):
    print(idx,val)

0 a
1 b
2 c
1 a
2 b
3 c
